# Machine Learning Engineer Nanodegree
## Deep Learning
## Project: Build a Digit Recognition Program

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

----
## Step 1: Design and Test a Model Architecture
Design and implement a deep learning model that learns to recognize sequences of digits. Train the model using synthetic data generated by concatenating character images from [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) or [MNIST](http://yann.lecun.com/exdb/mnist/). To produce a synthetic sequence of digits for testing, you can for example limit yourself to sequences up to five digits, and use five classifiers on top of your deep network. You would have to incorporate an additional ‘blank’ character to account for shorter number sequences.

There are various aspects to consider when thinking about this problem:
- Your model can be derived from a deep neural net or a convolutional network.
- You could experiment sharing or not the weights between the softmax classifiers.
- You can also use a recurrent network in your deep neural net to replace the classification layers and directly emit the sequence of digits one-at-a-time.

You can use ** Keras ** to implement your model. Read more at [keras.io](https://keras.io/).

Here is an example of a [published baseline model on this problem](http://static.googleusercontent.com/media/research.google.com/en//pubs/archive/42241.pdf). ([video](https://www.youtube.com/watch?v=vGPI_JvLoN0)). You are not expected to model your architecture precisely using this model nor get the same performance levels, but this is more to show an exampe of an approach used to solve this particular problem. We encourage you to try out different architectures for yourself and see what works best for you. Here is a useful [forum post](https://discussions.udacity.com/t/goodfellow-et-al-2013-architecture/202363) discussing the architecture as described in the paper.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [1]:
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, merge
from keras.layers import Input, Convolution2D, MaxPooling2D
from keras.utils.np_utils import to_categorical

import os
import numpy as np
from six.moves import cPickle as pickle
from PIL import Image
from scipy.misc import imresize

Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

((60000, 28, 28), (60000,))
((10000, 28, 28), (10000,))


In [39]:
sample = np.array(X_train[np.random.randint(X_train.shape[0]), :, :])

image = Image.fromarray(sample, 'L')
image.show()

In [3]:
# Don't run unless generating new data!

num_labels = 11
image_size = 28

def generateNewDataset(source_data, source_labels, sequences, max_sequence_length=5):

    new_dataset = np.ndarray((sequences, image_size, image_size), dtype=np.int8)
    # new_dataset = np.ndarray((sequences, image_size, image_size * max_sequence_length), dtype=np.int8)
    new_labels = np.ndarray((sequences, num_labels, max_sequence_length), dtype=np.int8)
    sequence_lengths = np.ndarray((sequences, max_sequence_length), dtype=np.int8)
    
    blank_label = np.matrix([0] * 10 + [1]).transpose(1, 0)
    # blank_image = np.matrix([[0] * image_size] * image_size)
    for sequence in range(sequences):
        
        # Randomly choose sequence length and sample indices
        sequence_length = np.random.randint(1, max_sequence_length + 1)
        sequence_lengths[sequence, sequence_length - 1] = 1
        sample_indices = []
        for bar in range(sequence_length):
            sample_indices.append(np.random.randint(source_data.shape[0]))

        # Pull the data and labels from the original sources and concatenate
        sample = np.matrix(source_data[sample_indices[0], :, :])
        label = np.matrix([0] * 11).transpose(1, 0)
        label[source_labels[sample_indices[0]]] = 1
        for character in sample_indices[1:]:
            sample = np.concatenate((sample, np.matrix(source_data[character, :, :])), axis=1)
            temp_label = np.matrix([0] * 11).transpose(1, 0)
            temp_label[source_labels[character]] = 1
            label = np.concatenate((label, temp_label), axis=1)
            
        # Resize sequence image to constant width
        new_image = imresize(sample, (image_size, image_size), interp='nearest')
        
        # Buffer label data to make all samples the same shape
        for bar in range(max_sequence_length - sequence_length):
            # sample = np.concatenate((sample, blank_image), axis=1)
            label = np.concatenate((label, blank_label), axis=1)
            
        # Append current sample to new data and label sets
        new_dataset[sequence, :, :] = new_image
        new_labels[sequence, :, :] = label
    return new_dataset, new_labels, sequence_lengths

new_train_dataset, new_train_labels, train_sequences = generateNewDataset(X_train, y_train, 1000000)
new_test_dataset, new_test_labels, test_sequences = generateNewDataset(X_test, y_test, 30000)

print("Generating sequence data complete.")
print(new_train_dataset.shape, new_train_labels.shape, train_sequences.shape)
print(new_test_dataset.shape, new_test_labels.shape, test_sequences.shape)

Generating sequence data complete.
((1000000, 28, 28), (1000000, 11, 5), (1000000, 5))
((30000, 28, 28), (30000, 11, 5), (30000, 5))


In [66]:
sample = np.array(new_train_dataset[np.random.randint(new_train_dataset.shape[0]), :, :])

image = Image.fromarray(sample, 'L')
image.show()

In [4]:
# Don't run unless saving a new batch of data!

pickle_file = 'MNISTseq1Mb.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': new_train_dataset,
    'train_labels': new_train_labels,
    'train_sequences': train_sequences,
    'test_dataset': new_test_dataset,
    'test_labels': new_test_labels,
    'test_sequences': test_sequences,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise
    
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

('Compressed pickle size:', 869320420)


In [5]:
# Run this cell after the import statements if the sequence data is already pickled

pickle_file = 'MNISTseq1Mb.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  train_sequences = save['train_sequences']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  test_sequences = save['test_sequences']
  del save  # hint to help gc free up memory

image_size = 28
num_labels = 11
max_sequence_length=5
num_channels = 1 # grayscale

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, num_channels, image_size, image_size)).astype(np.float32)
  labels = labels.reshape(-1, num_labels, max_sequence_length)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape, train_sequences.shape)
print('Test set', test_dataset.shape, test_labels.shape, test_sequences.shape)

('Training set', (1000000, 1, 28, 28), (1000000, 11, 5), (1000000, 5))
('Test set', (30000, 1, 28, 28), (30000, 11, 5), (30000, 5))


In [ ]:
# Model adapted from "VGG-like convnet" at https://keras.io/getting-started/sequential-model-guide/

inputs = Input(shape=(num_channels, image_size, image_size))
c1 = Convolution2D(32, 3, 3, border_mode='same')(inputs)
a1 = Activation('relu')(c1)
c2 = Convolution2D(32, 3, 3, border_mode='same')(a1)
a2 = Activation('relu')(c2)
mp1 = MaxPooling2D(pool_size=(2, 2), border_mode='same')(a2)
do1 = Dropout(0.25)(mp1)

c3 = Convolution2D(64, 3, 3, border_mode='same')(do1)
a3 = Activation('relu')(c3)
c4 = Convolution2D(64, 3, 3, border_mode='same')(a3)
a4 = Activation('relu')(c4)
mp2 = MaxPooling2D(pool_size=(2, 2), border_mode='same')(a4)
do2 = Dropout(0.25)(mp2)

f1 = Flatten()(do2)
d1 = Dense(256)(f1)
a5 = Activation('relu')(d1)
do3 = Dropout(0.5)(a5)

d2 = Dense(max_sequence_length)(do3)
a6 = Activation('softmax')(d2) # <--- Sequence length representation

m1 = merge([do3, a6], mode='concat')

d4 = Dense(num_labels)(m1)
o1 = Activation('softmax')(d4) # First character/digit
d5 = Dense(num_labels)(m1)
o2 = Activation('softmax')(d5)
d6 = Dense(num_labels)(m1)
o3 = Activation('softmax')(d6)
d7 = Dense(num_labels)(m1)
o4 = Activation('softmax')(d7)
d8 = Dense(num_labels)(m1)
o5 = Activation('softmax')(d8) # Fifth character/digit

# Worth considering:
# a LSTM will transform the vector sequence into a single vector,
# containing information about the entire sequence
# lstm_out = LSTM(output_length)(prior_layer)

clf = Model(input=inputs, output=[a6, o1, o2, o3, o4, o5]) # Seq length + five channel output

clf.compile(loss='categorical_crossentropy', optimizer='adadelta', 
            metrics=['accuracy']) # , loss_weights=[1., 1., 0.8, 0.6, 0.4, 0.2])

clf.fit(train_dataset, [train_sequences, train_labels[:,:,0], train_labels[:,:,1], train_labels[:,:,2], 
                        train_labels[:,:,3], train_labels[:,:,4]], batch_size=128, 
                        nb_epoch=2, verbose=1, validation_split=0.03)

clf.evaluate(test_dataset, [test_sequences, test_labels[:,:,0], test_labels[:,:,1], test_labels[:,:,2], 
                        test_labels[:,:,3], test_labels[:,:,4]], verbose=1)

# pred = clf.predict(test_dataset)

Train on 970000 samples, validate on 30000 samples
Epoch 1/2
970000/970000 [==============================] - 302s - loss: 459.4824 - activation_29_loss: 452.5064 - activation_30_loss: 2.2842 - activation_31_loss: 1.8524 - activation_32_loss: 1.4008 - activation_33_loss: 0.9491 - activation_34_loss: 0.4896 - activation_29_acc: 0.2620 - activation_30_acc: 0.1263 - activation_31_acc: 0.2897 - activation_32_acc: 0.4640 - activation_33_acc: 0.6396 - activation_34_acc: 0.8184 - val_loss: 457.2940 - val_activation_29_loss: 450.5562 - val_activation_30_loss: 2.1748 - val_activation_31_loss: 1.8045 - val_activation_32_loss: 1.3743 - val_activation_33_loss: 0.9239 - val_activation_34_loss: 0.4603 - val_activation_29_acc: 0.3298 - val_activation_30_acc: 0.2248 - val_activation_31_acc: 0.3263 - val_activation_32_acc: 0.4844 - val_activation_33_acc: 0.6463 - val_activation_34_acc: 0.8220
Epoch 2/2
564352/970000 [================>.............] - ETA: 129s - loss: 457.8524 - activation_29_loss: 451

### Question 1
_What approach did you take in coming up with a solution to this problem?_

**Answer:** I first designed a multilayer convolutional network, based on the Keras documentation, and made certain it performed well on single digits from the Keras MNIST dataset (98.57% accuracy on test set). 

Then I tested a deeper network on the notMNIST dataset (95.42% accuracy on the test set). Validation accuracy appears to plateau after 12 Epochs of 200,000 samples in batches of 128.

Next I modified the data sets to use random combinations of up to five samples from the notMNIST data and added parallel softmax outputs in a similar fashion to the Google architecture linked above.

### Question 2
_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)_

**Answer:**

### Question 3
_How did you train your model? How did you generate your synthetic dataset?_ Include examples of images from the synthetic data you constructed.

**Answer:**

----
## Step 2: Train a Model on a Realistic Dataset
Once you have settled on a good architecture, you can train your model on real data. In particular, the [Street View House Numbers (SVHN)](http://ufldl.stanford.edu/housenumbers/) dataset is a good large-scale dataset collected from house numbers in Google Street View. Training on this more challenging dataset, where the digits are not neatly lined-up and have various skews, fonts and colors, likely means you have to do some hyperparameter exploration to perform well.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 4
_Describe how you set up the training and testing data for your model. How does the model perform on a realistic dataset?_

**Answer:**

### Question 5
_What changes did you have to make, if any, to achieve "good" results? Were there any options you explored that made the results worse?_

**Answer:**

### Question 6
_What were your initial and final results with testing on a realistic dataset? Do you believe your model is doing a good enough job at classifying numbers correctly?_

**Answer:**

----
## Step 3: Test a Model on Newly-Captured Images

Take several pictures of numbers that you find around you (at least five), and run them through your classifier on your computer to produce example results. Alternatively (optionally), you can try using OpenCV / SimpleCV / Pygame to capture live images from a webcam and run those through your classifier.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 7
_Choose five candidate images of numbers you took from around you and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult?_

**Answer:**

### Question 8
_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the realistic dataset?_

**Answer:**

### Optional: Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._

**Answer:** Leave blank if you did not complete this part.

----
### Step 4: Explore an Improvement for a Model

There are many things you can do once you have the basic classifier in place. One example would be to also localize where the numbers are on the image. The SVHN dataset provides bounding boxes that you can tune to train a localizer. Train a regression loss to the coordinates of the bounding box, and then test it. 

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 10
_How well does your model localize numbers on the testing set from the realistic dataset? Do your classification results change at all with localization included?_

**Answer:**

### Question 11
_Test the localization function on the images you captured in **Step 3**. Does the model accurately calculate a bounding box for the numbers in the images you found? If you did not use a graphical interface, you may need to investigate the bounding boxes by hand._ Provide an example of the localization created on a captured image.

**Answer:**

----
## Optional Step 5: Build an Application or Program for a Model
Take your project one step further. If you're interested, look to build an Android application or even a more robust Python program that can interface with input images and display the classified numbers and even the bounding boxes. You can for example try to build an augmented reality app by overlaying your answer on the image like the [Word Lens](https://en.wikipedia.org/wiki/Word_Lens) app does.

Loading a TensorFlow model into a camera app on Android is demonstrated in the [TensorFlow Android demo app](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/android), which you can simply modify.

If you decide to explore this optional route, be sure to document your interface and implementation, along with significant results you find. You can see the additional rubric items that you could be evaluated on by [following this link](https://review.udacity.com/#!/rubrics/413/view).

### Optional Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your optional code implementation goes here.
### Feel free to use as many code cells as needed.



### Documentation
Provide additional documentation sufficient for detailing the implementation of the Android application or Python program for visualizing the classification of numbers in images. It should be clear how the program or application works. Demonstrations should be provided. 

_Write your documentation here._

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.